# Exercise; using Pyomo

- GLPK, CBC installed with brew
- Pyomo installed with poetry
- Gurobi licence installed with: `% grbgetkey 8856222a-c3ea-11ed-85be-0242ac120002`

- CPlex Downloaded from here: https://ibm.ent.box.com/s/wjuh81fmorssmwwoe4eg2spfl9xrakvn (using Safari
    - Activate for Python here: % python /Applications/CPLEX_Studio_Community2211/python/setup.py install

We wish to minimise -4x - 2y, given these constraints:

x + y <= 8  
8x + 3y <= -24
-6x + 8y <= 48  
3x + 5y <= 15


x <= 3  
0 <= y

In [1]:
#!/usr/bin/python
import os, sys
# Add the directory containing the SCIP shared libraries to the DYLD_LIBRARY_PATH variable
scip_shared_lib_path = "/usr/local/scipoptsuite-8.0.3/scip/lib/shared"
if "DYLD_LIBRARY_PATH" not in os.environ:
    print(f"DYLD_LIBRARY_PATH not set, setting to {scip_shared_lib_path}")
    os.environ["DYLD_LIBRARY_PATH"] = scip_shared_lib_path

if "/usr/local/scipoptsuite-8.0.3/scip/lib/shared" not in os.environ["DYLD_LIBRARY_PATH"]:
    print(f"Adding {scip_shared_lib_path} to DYLD_LIBRARY_PATH")
    os.environ["DYLD_LIBRARY_PATH"] = scip_shared_lib_path + ":" + os.environ["DYLD_LIBRARY_PATH"]

# Check that the SCIP executable exists (correct for your system)
scip_executable = "/usr/local/scipoptsuite-8.0.3/scip/bin/scip"
print("SCIP executable exists:", os.path.isfile(scip_executable))
cbc_executable = "/opt/homebrew/Cellar/cbc/2.10.8/bin/cbc"
print("CBC executable exists:", os.path.isfile(cbc_executable))
cplex_executable_arm = "/Applications/CPLEX_Studio_Community2211/cplex/bin/arm64_osx/cplex"
print("CPLEX executable exists:", os.path.isfile(cplex_executable_arm))


import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

# Create a model
model = pyo.ConcreteModel()

# Create the variables
# x <= 3  
# 0 <= y
model.x = pyo.Var(bounds=(None, 3))
model.y = pyo.Var(bounds=(0, None))

x = model.x
y = model.y



SCIP executable exists: True
CBC executable exists: True
CPLEX executable exists: True


In [2]:
# Type in our constraints
# x + y <= 8  
# 8x + 3y <= -24
# -6x + 8y <= 48  
# 3x + 5y <= 15
model.C1 = pyo.Constraint(expr= x + y <= 8)
model.C2 = pyo.Constraint(expr= 8*x + 3*y <= -24)
model.C3 = pyo.Constraint(expr= -6*x + 8*y <= 48)
model.C4 = pyo.Constraint(expr= 3*x + 5*y <= 15)

# Type in our objective function
# We wish to minimise -4x - 2y
model.obj = pyo.Objective(expr= -4*x - 2*y, sense=minimize)

In [3]:
 # print out the PATH, LD_LIBRARY_PATH and DYLD_LIBRARY_PATH environment variables
        # separate by colons
# pprint.pprint(f"PATH = \n{os.environ['PATH'].split(':')}")
# pprint.pprint(f"DYLD_LIBRARY_PATH = \n{os.environ['DYLD_LIBRARY_PATH'].split(':')}")

In [4]:
# Solve the model, comparing the speed of Gurobi, GLPK, CPLEX, SCIP and CBC solvers
# Use the package time to measure the time taken to solve the model

import os
import pandas as pd
from pyomo.environ import SolverFactory
# ... (Your other imports and model definition)

def configure_solvers():
    solvers = [
        {
            "name": "gurobi",
            "executable": "",
            "path": "",
        },
        {
            "name": "glpk",
            "executable": "",
            "path": "",
        },
        {
            "name": "cplex",
            "executable": "",
            "path": "/Applications/CPLEX_Studio_Community2211/cplex/bin/arm64_osx",
        },
        {
            "name": "scip",
            "executable": "/usr/local/scipoptsuite-8.0.3/scip/bin/scip-8.0.3.darwin.arm.gnu.opt.spx2.none",  # Full path to the SCIP executable
            "path": "/usr/local/scipoptsuite-8.0.3/scip/bin",
            "shared_library_path": "/usr/local/scipoptsuite-8.0.3/scip/lib/shared",
            "soplex_path": "/usr/local/scipoptsuite-8.0.3/soplex/lib/shared",
        },

        {
            "name": "cbc",
            "executable": "",
            "path": "/opt/homebrew/Cellar/cbc/2.10.8/bin",
        },
    ]

    for solver in solvers:
        if solver["executable"]:
            executable_name = solver["executable"]
        else:
            executable_name = solver["name"]

        if solver["path"] and executable_name not in os.environ["PATH"]:
            os.environ["PATH"] = f"{solver['path']}:{os.environ['PATH']}"

        if solver.get("shared_library_path") and solver["shared_library_path"] not in os.environ.get("DYLD_LIBRARY_PATH", ""):
            os.environ["DYLD_LIBRARY_PATH"] = f"{solver['shared_library_path']}:{os.environ.get('DYLD_LIBRARY_PATH', '')}"

        if solver.get("soplex_path") and solver["soplex_path"] not in os.environ.get("DYLD_LIBRARY_PATH", ""):
            os.environ["DYLD_LIBRARY_PATH"] = f"{solver['soplex_path']}:{os.environ.get('DYLD_LIBRARY_PATH', '')}"

configure_solvers()

results_df = pd.DataFrame(columns=['Solver', 'x', 'y', 'Time'])

for solver_name in ['gurobi', 'glpk', 'cplex', 'scip', 'cbc']:
    try: 
        opt = SolverFactory(solver_name)
        results = opt.solve(model)
        print (f"Results for {solver_name} solver: x = {pyo.value(x)}, y = {pyo.value(y)}")
        print(f"{solver_name} took {results.solver.time} seconds to solve the model")
        results.solver.time = round(results.solver.time, 4)
        results_df = pd.concat([results_df, pd.DataFrame({'Solver': solver_name, 'x': pyo.value(x), 'y': pyo.value(y), 'Time': results.solver.time}, index=[0])])
    except Exception as e:
        # provide useful error message to debug, using PATH and DYLD_LIBRARY_PATH (relevant lines only)
        # filter for SCIP entries
        filtered_path = [x for x in os.environ['PATH'].split(':') if 'scip' in x]
        print(f"PATH = \n{filtered_path}")
        filtered_dyld = [x for x in os.environ['DYLD_LIBRARY_PATH'].split(':') if 'scip' in x]
        print(f"DYLD_LIBRARY_PATH = \n{filtered_dyld}")
        
        print(f"Error: {e}")
        print(f"{solver_name} solver not available")
        continue

print(f"Results: \n======= \n")

from IPython.display import display
results_df = results_df.sort_values(by=['Time'])
display(results_df)



       
        





Results for gurobi solver: x = -4.097560975609756, y = 2.926829268292683
gurobi took 0.9378170967102051 seconds to solve the model
Results for glpk solver: x = -4.09756097560976, y = 2.92682926829268
glpk took 0.02432394027709961 seconds to solve the model
Results for cplex solver: x = -4.097560975609756, y = 2.926829268292683
cplex took 0.269428014755249 seconds to solve the model
Results for scip solver: x = -4.097560975609756, y = 2.926829268292683
scip took 0.0 seconds to solve the model
Results for cbc solver: x = -4.097561, y = 2.9268293
cbc took 1.326596975326538 seconds to solve the model
Results: 



,Solver,x,y,Time
0,scip,-4.097561,2.926829,0.0000
0,glpk,-4.097561,2.926829,0.0243
0,cplex,-4.097561,2.926829,0.2694
0,gurobi,-4.097561,2.926829,0.9378
0,cbc,-4.097561,2.926829,1.3266


In [6]:
# import subprocess
# import os

# scip_command = "/usr/local/scipoptsuite-8.0.3/scip/bin/scip-8.0.3.darwin.arm.gnu.opt.spx2.none -f my_model.lp"

# # Set environment variables
# my_env = os.environ.copy()
# scip_shared_lib_path = "/usr/local/scipoptsuite-8.0.3/scip/lib/shared"
# scip_soplex_path = "/usr/local/scipoptsuite-8.0.3/soplex/lib/shared"
# my_env["DYLD_LIBRARY_PATH"] = f"{scip_shared_lib_path}:{scip_soplex_path}:{my_env.get('DYLD_LIBRARY_PATH', '')}"
# my_env["PATH"] = f"{my_env['PATH']}:/usr/local/scipoptsuite-8.0.3/scip/bin"

# output = subprocess.check_output(scip_command, shell=True, text=True, env=my_env)
